In [ ]:
%cd /content
!git clone https://github.com/aimacode/aima-python.git 
%cd /content/aima-python/
!git clone https://github.com/aimacode/aima-data.git
!pip install ipythonblocks qpsolvers

In [1]:
from agents import *
from notebook import psource

# CONTENTS

1. Overview
   >1.1 Agent   
   >1.2 Environment   
      
2. Simple Agent and Environment
   > 2.1 Environment-park   
   > 2.2 Program - BlindDog   
   > 2.3 Execution   
      
3. Agents in a 2-D Environment
   > 3.1 BlindDog in Graph2D Environment    
   > 3.2 EnergeticBlindDog in Graph2D Environment
4. Vacuum Cleaner Environment
   > 4.0 TrivialVacuumEnvironment   
   > 4.1 Random Agent Program   
   > 4.2 Table-Driven Agent Program   
   > 4.3 Simple Reflex Agent Program   
   > 4.4 Model-Based Reflex Agent Program   

# 1. Overview

## 1.1 AGENT

In [2]:
psource(Environment)

## 1.2 ENVIRONMENT

In [3]:
psource(Environment)

# 3. AGENTS IN A 2D ENVIRONMENT

In [5]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''에이전트 위치에있는 항목 목록 반환'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''에이전트가 수행하는 작업에 따라 환경 상태 변경'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''기본적으로 우리는 살아있는 요원을 찾을 수 없을 때 완료되지만, 귀여운 강아지를 죽이는 것을 방지하기 위해 음식이나 물이 더 이상 없을 때 스스로 중지'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [6]:
class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing # 방향 정해주는 것
    
    def movedown(self):
        self.location[1] += 1 # move 다운으로 하면 location을 변화시켜줌
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

In [43]:
def program(percepts):
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

### 3.1 BlindDog in Graph2D Environment

### >park 환경 선언 (너비: 5 / 길이:20 / BlindDog의 색: 빨간색 / Water의 색: 파란색 / Food의 색: 주황색

In [44]:
park = Park2D(width = 1, height = 20, display = True, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)}) # park width is set to 5, and height to 20

""
""
""
""
""
""
""
""
""
""
""


### >Park에 agent, food, water 넣어줌

In [45]:
dog = BlindDog(program)  
dogfood = Food()  
water = Water()  
park.add_thing(dog, [0,1]) 
park.add_thing(dogfood, [0,5]) 
park.add_thing(water, [0,7]) 

morewater = Water()
park.add_thing(morewater, [0,15]) 

### >20 step 실시

In [46]:
steps = 20
park.run(steps)

""
""
""
""
""
""
""
""
""
""
""


### 3.2 EnergeticBlindDog in Graph2D Environment

## PROGRAM - EnergeticBlindDog

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

### >단방향이 아닌 여러 방향으로 움직이는 BlindDog

In [11]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down") 
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

In [12]:
def program(percepts):
    '''Returns an action based on it's percepts'''       
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'

        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));  # 1-right, 2-left, others-forward
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward

    # choice 1이면 오른쪽으로 2이면 왼쪽으로 아니면 기본 방향 대로

    if choice == 1:
        return 'turnright'
    
    elif choice == 2:
        return 'turnleft'
    
    else:
        return 'moveforward'

### ENVIRONMENT - Park2D

In [13]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1

        if not self.is_inbounds(loc):
            things.append(Bump())        
        return things

    def execute_action(self, agent, action):
        '''에이전트가 수행하는 작업에 따라 환경 상태를 변경'''
        
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
            
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        
        elif action == 'moveforward':
            print(agent.direction.direction)
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        
        elif action == "eat": #location을 바꿔주고 thing을 없앰
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
        elif action == "drink": #location을 바꿔주고 thing을 없앰
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print(f'{str(agent)[1:-1]} drank {str(items[0])[1:-1]} at location: {agent.location}')
                    self.delete_thing(items[0])
                    
    def is_done(self): #종료 조건
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

### >park 환경 선언 (너비: 5 / 길이:20 / BlindDog의 색: 빨간색 / Water의 색: 주황색 / Food의 색: 파란색

In [14]:
park = Park2D(5,15, display = True, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


### >Park에 들어갈 EnergeticBlindDog, Food, Water 각각의 객체를 선언

In [15]:
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()

park.add_thing(dog, [0,1])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])

morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])

### >20 step 실시

In [16]:
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


# 4. THE VACUUM WORLD   

## 4.0 Trivial Vacuum  Environment

In [47]:
psource(TrivialVacuumEnvironment)

## 4.1 Random Agent Program

In [38]:
trivial_vacuum_env = TrivialVacuumEnvironment()

random_agent = Agent(program=RandomAgentProgram(['Right', 'Left', 'Suck', 'NoOp']))

print("초기 환경의 상태: {}.".format(trivial_vacuum_env.status))
trivial_vacuum_env.add_thing(random_agent)
print("초기 에이전트의 위치: {}.".format(random_agent.location), '\n')

trivial_vacuum_env.step()
print("1 step 후 환경의 상태: {}.".format(trivial_vacuum_env.status))
print("1 step 후 에이전트의 현재 위치 {}.".format(random_agent.location))

초기 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}.
초기 에이전트의 위치: (0, 0). 

1 step 후 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}.
1 step 후 에이전트의 현재 위치 (1, 0).


## 4.2 TABLE-DRIVEN AGENT PROGRAM

In [39]:
table = {((loc_A, 'Clean'),): 'Right',
             ((loc_A, 'Dirty'),): 'Suck',
             ((loc_B, 'Clean'),): 'Left',
             ((loc_B, 'Dirty'),): 'Suck',
             ((loc_A, 'Dirty'), (loc_A, 'Clean')): 'Right',
             ((loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
             ((loc_B, 'Clean'), (loc_A, 'Dirty')): 'Suck',
             ((loc_B, 'Dirty'), (loc_B, 'Clean')): 'Left',
             ((loc_A, 'Dirty'), (loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
             ((loc_B, 'Dirty'), (loc_B, 'Clean'), (loc_A, 'Dirty')): 'Suck'
        }

In [40]:
trivial_vacuum_env = TrivialVacuumEnvironment()
table_driven_agent = Agent(program=TableDrivenAgentProgram(table=table))

print("초기 환경의 상태: {}.".format(trivial_vacuum_env.status))
trivial_vacuum_env.add_thing(table_driven_agent)
print("초기 에이전트 위치 {}.".format(table_driven_agent.location), '\n')

trivial_vacuum_env.step()
print("1 step 후 환경의 상태: {}.".format(trivial_vacuum_env.status))
print("1 step 후 현재 에이전트의 위치 {}.".format(table_driven_agent.location))

초기 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}.
초기 에이전트 위치 (0, 0). 

1 step 후 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}.
1 step 후 현재 에이전트의 위치 (1, 0).


## 4.3 SIMPLE REFLEX AGENT PROGRAM

In [41]:
trivial_vacuum_env = TrivialVacuumEnvironment()


def SimpleReflexAgentProgram():  
    def program(percept):
        loc, status = percept
        return ('Suck' if status == 'Dirty' else'Right' if loc == loc_A else'Left')
    return program
        
program = SimpleReflexAgentProgram()
simple_reflex_agent = Agent(program)

trivial_vacuum_env.add_thing(simple_reflex_agent)
print("초기 에이전트 위치: {}.".format(simple_reflex_agent.location))
print("초기 환경의 상태: {}.".format(trivial_vacuum_env.status), '\n')

trivial_vacuum_env.step()
print("1 step 후 에이전트의 위치 {}.".format(simple_reflex_agent.location))
print("1 step 후 환경의 상태: {}.".format(trivial_vacuum_env.status))

초기 에이전트 위치: (1, 0).
초기 환경의 상태: {(0, 0): 'Dirty', (1, 0): 'Clean'}. 

1 step 후 에이전트의 위치 (0, 0).
1 step 후 환경의 상태: {(0, 0): 'Dirty', (1, 0): 'Clean'}.


## 4.4 MODEL-BASED REFLEX AGENT PROGRAM

In [42]:
trivial_vacuum_env = TrivialVacuumEnvironment()

def ModelBasedVacuumAgent():
    model = {loc_A: None, loc_B: None}
    def program(percept):
        location, status = percept
        model[location] = status  # Update the model here
        if model[loc_A] == model[loc_B] == 'Clean':
            return 'NoOp'
        elif status == 'Dirty':
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Left'

    return Agent(program)

model_based_reflex_agent = ModelBasedVacuumAgent()

trivial_vacuum_env.add_thing(model_based_reflex_agent)
print("초기 에이전트의 위치 {}.".format(model_based_reflex_agent.location))
print("초기 환경의 상태: {}.".format(trivial_vacuum_env.status), '\n')


trivial_vacuum_env.step()
print("1 스텝 후 에이전트의 현재 위치: {}.".format(model_based_reflex_agent.location))
print("1 스텝 후 환경의 상태: {}.".format(trivial_vacuum_env.status))

초기 에이전트의 위치 (0, 0).
초기 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}. 

1 스텝 후 에이전트의 현재 위치: (1, 0).
1 스텝 후 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}.
